In [2275]:
import pandas as pd
import numpy as np
import os

In [2276]:
#pip install pandas

In [2277]:
# Helper Methods

def generate_column_info(df):
    cls = []
    nullCount = []
    nonNullCount = []
    nullsPct = []
    uniqCount = []
    dataType = []
    for i,col in enumerate(df.columns):
        cls.append(col)
        nullCount.append(df[col].isnull().sum())
        nonNullCount.append(len(df)-df[col].isnull().sum())
        nullsPct.append((df[col].isnull().sum())*(100)/len(df))
        uniqCount.append(df[col].nunique())
        dataType.append(df[col].dtype)
        
    column_info = pd.DataFrame(
        {'ColumnName': cls,
         'NullCount': nullCount,
         'NonNullCount': nonNullCount,
         'NullPercent': nullsPct,
         'UniqueValueCount': uniqCount,
         'DataType':dataType
        })
    return(column_info)

## Data

abbr_dict = {
    'flight_Liquefied Natural Gas ImpExp Equipment.xls':'LNG_IMEX',
    'flight_Liquefied Natural Gas Storage.xls':'LNG_STOR',
    'flight_Natural Gas Local Distribution Companies.xls':'NG_LDC',
    'flight_Natural Gas Processing.xls':'NG_PROC',
    'flight_Natural Gas TransmissionCompression.xls':'NG_COMP',
    'flight_Offshore Petroleum & Natural Gas Production.xls':'PROD_OFF',
    'flight_Onshore Natural Gas Transmission Pipelines.xls':'NG_PIPE',
    'flight_Onshore Petroleum & Natural Gas Gathering & Boosting.xls':'NG_GNB',
    'flight_Onshore Petroleum & Natural Gas Production.xls':'PROD_ON',
    'flight_Other Petroleum & Natural Gas Systems.xls':'ONG_OTH',
    'flight_Underground Natural Gas Storage.xls':'NG_STOR',
}

sub_sector_dict = {
    'NG_LDC':'NaturalGasDistribution',
    'PROD_OFF':'OffshoreProduction',
    'PROD_ON':'OnshoreProduction',
}

In [2278]:
#pip install pyxlsb

## Facilities List

In [2279]:
#directory_path = 'C://Users//shashank.bekal//PythonWorkArea//Jnana-Research//OKC-Facilities//files'
directory_path = './/files'
all_files = os.listdir(directory_path)
xls_files = [file for file in all_files if file.endswith('.xls')]

facilities_master_df = pd.DataFrame()
for file_name in xls_files:
    file_dict = pd.read_excel(f'.//files/{file_name}',sheet_name=None,skiprows=6)
    file_df = pd.concat(file_dict, ignore_index=True)
    file_df["Abbr"] = abbr_dict[file_name]
    #print((file_name))
    facilities_master_df = pd.concat([facilities_master_df,file_df])
#print('---')
print(len(facilities_master_df))

C:\Users\shankze\AppData\Local\Temp\ipykernel_8412\1785748913.py:9: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  file_df = pd.concat(file_dict, ignore_index=True)
C:\Users\shankze\AppData\Local\Temp\ipykernel_8412\1785748913.py:9: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  file_df = pd.concat(file_dict, ignore_index=True)
C:\Users\shankze\AppData\Local\Temp\ipykernel_8412\1785748913.py:9: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a 

30781


C:\Users\shankze\AppData\Local\Temp\ipykernel_8412\1785748913.py:9: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  file_df = pd.concat(file_dict, ignore_index=True)
C:\Users\shankze\AppData\Local\Temp\ipykernel_8412\1785748913.py:9: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  file_df = pd.concat(file_dict, ignore_index=True)


In [2280]:
required_subsectors = ['PROD_OFF','PROD_ON']

# filter by 3 required subsectors
facilities_master_df = facilities_master_df[facilities_master_df['Abbr'].isin(required_subsectors)]
facilities_master_df['sub_sector'] = facilities_master_df['Abbr'].map(sub_sector_dict)
facilities_master_df = facilities_master_df.rename(columns={'GHG QUANTITY (METRIC TONS CO2e)':'FACILITY_WIDE_GHG_QUANTITY'})
facilities_master_df = facilities_master_df.rename(columns={'GHGRP ID':'facility_id','REPORTING YEAR':'reporting_year'})
facilities_master_df = facilities_master_df.drop(['FACILITY NAME','SUBPARTS'],axis=1)

In [2281]:
#facilities_master_df['sub_sector'].value_counts()

In [2282]:
generate_column_info(facilities_master_df)

,ColumnName,NullCount,NonNullCount,NullPercent,UniqueValueCount,DataType
0,reporting_year,0,8055,0.000000,13,object
1,facility_id,0,8055,0.000000,1149,object
2,REPORTED ADDRESS,1645,6410,20.422098,746,object
3,LATITUDE,0,8055,0.000000,786,float64
4,LONGITUDE,0,8055,0.000000,784,float64
5,CITY NAME,0,8055,0.000000,191,object
6,COUNTY NAME,5348,2707,66.393544,166,object
7,STATE,0,8055,0.000000,22,object
8,ZIP CODE,0,8055,0.000000,268,object
9,PARENT COMPANIES,42,8013,0.521415,1284,object


## EPA Verified Facilities List

In [2283]:
epa_ver_df = pd.read_excel('EPA_verified_facilities.xls',sheet_name=None,skiprows=6)
epa_ver_df = pd.concat(epa_ver_df, ignore_index=True)
epa_ver_df['REPORTING YEAR'] = epa_ver_df['REPORTING YEAR'].astype(int)
epa_ver_df['GHGRP ID'] = epa_ver_df['GHGRP ID'].astype(int)
epa_ver_df = epa_ver_df.rename(columns={'GHGRP ID':'facility_id','REPORTING YEAR':'reporting_year'})
epa_verified_unique_facilities_df = df[['facility_id', 'reporting_year']].drop_duplicates()
epa_verified_unique_facilities_df['EPA_verified'] = 1
epa_verified_unique_facilities_df

C:\Users\shankze\AppData\Local\Temp\ipykernel_8412\1253078419.py:2: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  epa_ver_df = pd.concat(epa_ver_df, ignore_index=True)


,facility_id,reporting_year,EPA_verified
0,1002685,2022,1
1,1002685,2021,1
2,1002685,2020,1
3,1002685,2019,1
4,1002685,2018,1
...,...,...,...
45100,1011290,2015,1
45997,1011724,2015,1
46867,1011289,2015,1
47127,1010485,2015,1


## Facilities Metadata

In [2284]:
# facilities_metadata_df = pd.read_csv('rlps_ghg_emitter_facilities.csv')
# facilities_metadata_df = facilities_metadata_df[['facility_id','year','epa_verified']]
# facilities_metadata_df

In [2285]:
# facilities_metadata_df.columns

In [2286]:
# facilities_metadata_df[facilities_metadata_df.duplicated(subset=['facility_id','year'])]

In [2287]:
# facilities_master_df = pd.merge(facilities_master_df,facilities_metadata_df,how='left',left_on=['GHGRP ID','REPORTING YEAR'],right_on=['facility_id','year'])

In [2288]:
# facilities_master_df

In [2289]:
# facilities_master_df.columns

In [2290]:
# generate_column_info(facilities_master_df)

## Combustion Sources

In [ ]:
#emissions_by_unit_df = pd.read_excel('emissions_by_unit_and_fuel_type_c_d_aa.xlsx',engine="openpyxl", sheet_name="UNIT_DATA")
c_df = pd.read_excel('emissions_by_unit_and_fuel_type_c_d_aa.xlsx', sheet_name="UNIT_DATA",skiprows=6)
c_df['Reporting Year'] = c_df['Reporting Year'].astype(int)
c_df['Facility Id'] = c_df['Facility Id'].astype(int)
c_df = c_df.rename(columns={'Facility Id':'facility_id','Facility Name':'facility_name','Reporting Year':'reporting_year'})
c_df = c_df.rename(columns={'Unit Type':'reporting_category','Unit CO2 emissions (non-biogenic) ':'total_co2_emissions','Unit Methane (CH4) emissions ':'total_ch4_emissions'})
c_df = c_df.rename(columns={'Unit Nitrous Oxide (N2O) emissions ':'total_n2o_emissions'})
c_df = c_df.drop(columns=['Unit Reporting Method','City','Primary NAICS Code','FRS Id','Industry Type (subparts)','Industry Type (sectors)','Unit Biogenic CO2 emissions (metric tons)'])
c_df = c_df.rename(columns={'Unit Maximum Rated Heat Input Capacity (mmBTU/hr)':'maximum_rated_heat_input_capacity','State':'state'})
#c_df = c_df[c_df['reporting_category'].notna()]
c_df['subpart'] = 'SubpartC'
c_df[['total_co2_emissions','total_ch4_emissions','total_n2o_emissions']] = c_df[['total_co2_emissions','total_ch4_emissions','total_n2o_emissions']].fillna(0)
c_df

# Emission Source

In [ ]:
w_df = pd.read_csv('ef_w_emissions_source_ghg.csv',encoding = 'ISO-8859-1')
print(len(w_df))
w_df['reporting_year'] = w_df['reporting_year'].astype(int)
w_df['facility_id'] = w_df['facility_id'].astype(int)
w_df = w_df.drop(columns=['bamm_desc_source_summary','bamm_indicator_source_summary','industry_segment'], axis=1)
w_df = w_df.rename(columns={'total_reported_co2_emissions':'total_co2_emissions'})
w_df = w_df.rename(columns={'total_reported_ch4_emissions':'total_ch4_emissions'})
w_df = w_df.rename(columns={'total_reported_n2o_emissions':'total_n2o_emissions'})
w_df[['total_co2_emissions','total_ch4_emissions','total_n2o_emissions']] = w_df[['total_co2_emissions','total_ch4_emissions','total_n2o_emissions']].fillna(0)
w_df['total_ch4_emissions'] = w_df['total_ch4_emissions'] * 25
w_df['total_n2o_emissions'] = w_df['total_n2o_emissions'] * 298
w_df['subpart'] = 'SubpartW'
w_df

In [ ]:
# #w_df[(w_df['reporting_year']==2022) & (w_df['facility_id']==1000435)]
# temp_df = w_df[(w_df['reporting_year']==2022) & (w_df['facility_id']==1000435)]#[['reporting_category','total_co2_emissions','total_ch4_emissions','total_n2o_emissions','total_ghg_emissions']]
# temp_df[['total_co2_emissions','total_ch4_emissions','total_n2o_emissions']] = temp_df[['total_co2_emissions','total_ch4_emissions','total_n2o_emissions']].fillna(0)
# temp_df['total_ghg_emissions'] = temp_df['total_co2_emissions'] + temp_df['total_ch4_emissions'] + temp_df['total_n2o_emissions']
# temp_df[['reporting_category','total_co2_emissions','total_ch4_emissions','total_n2o_emissions','total_ghg_emissions']]
# temp_df = temp_df[temp_df['total_ghg_emissions']>0]
# temp_df[['reporting_category','total_co2_emissions','total_ch4_emissions','total_n2o_emissions','total_ghg_emissions']]

## ConCat c_df and w_df

In [ ]:
df = pd.concat([c_df,w_df], ignore_index=True)

In [ ]:
generate_column_info(df)

In [ ]:
# df['total_ghg_emissions'] = df['total_co2_emissions'] + df['total_ch4_emissions'] + df['total_n2o_emissions']
# df = df[df['total_ghg_emissions']>0]

In [ ]:
len(df)

In [ ]:
df[(df['facility_id']==1001727)&(df['reporting_year']==2023)]

## Rename Reporting Category

In [ ]:
reporting_category_names_df = pd.read_csv('reporting_categories.csv')
df = pd.merge(df, reporting_category_names_df, how='left',on=['reporting_category'])
df = df.drop('reporting_category',axis=1)
df = df.rename(columns={'REVISED_COLUMNNAME':'reporting_category'})

## validate before substitutions

In [ ]:
# facilities_in_required_subsectors = facilities_master_df['facility_id'].unique()

# validation_before_sub_df = df
# validation_before_sub_df = validation_before_sub_df[validation_before_sub_df.facility_id.isin(facilities_in_required_subsectors)]

# validation_before_sub_df['total_ghg_emissions'] = validation_before_sub_df['total_co2_emissions'] + validation_before_sub_df['total_ch4_emissions'] + validation_before_sub_df['total_n2o_emissions']
# validation_before_sub_df = validation_before_sub_df[['facility_id','reporting_year','total_co2_emissions','total_ch4_emissions','total_n2o_emissions','total_ghg_emissions']]
# validation_before_sub_df=validation_before_sub_df.groupby(['facility_id','reporting_year']).sum().reset_index()
# validation_before_sub_df = pd.merge(validation_before_sub_df,facilities_master_df[['facility_id','reporting_year','FACILITY_WIDE_GHG_QUANTITY']],how='left',on=['facility_id','reporting_year'])

# validation_before_sub_df = validation_before_sub_df[validation_before_sub_df['FACILITY_WIDE_GHG_QUANTITY']>0]
# validation_before_sub_df['diff'] = (validation_before_sub_df['FACILITY_WIDE_GHG_QUANTITY']-validation_before_sub_df['total_ghg_emissions']).abs()
# validation_before_sub_df['diff_percent_GHG']=100*validation_before_sub_df['diff']/validation_before_sub_df['FACILITY_WIDE_GHG_QUANTITY']
# validation_before_sub_df.to_csv('validation_before_sub_df.csv', index=False)
# validation_before_sub_df

In [ ]:
#validation_df[validation_df.diff_percent_GHG > 10]

## Concat Pneumatic Devices from detailed dataset

In [ ]:
len(df[df.reporting_category == 'NG_PneumaticDevices'])

In [ ]:
df.columns

In [ ]:
# Drop Pneumatic Devices from original dataset
df = df[df.reporting_category != 'NG_PneumaticDevices']

# add them from this csv file
pneumatic_devices_df = pd.read_csv('ef_w_ngpneumatic_dev_units.csv')
pneumatic_devices_df = pneumatic_devices_df[['facility_id','reporting_year','facility_name','estimated_number_of_hours','pneumatic_device_type','total_count','pneum_device_type_tot_co2_emm','total_ch4_emissions']]
pneumatic_devices_df=pneumatic_devices_df.rename(columns={'pneumatic_device_type':'Unit Name','estimated_number_of_hours':'pneu_est_no_of_hours','total_count':'pneu_dev_total_count','pneum_device_type_tot_co2_emm':'total_co2_emissions'})
pneumatic_devices_df[['total_co2_emissions','total_ch4_emissions']] = pneumatic_devices_df[['total_co2_emissions','total_ch4_emissions']].fillna(0)
pneumatic_devices_df['total_ch4_emissions'] = pneumatic_devices_df['total_ch4_emissions'] * 25
pneumatic_devices_df['total_n2o_emissions'] = 0
pneumatic_devices_df['reporting_category'] = 'NG_PneumaticDevices'
pneumatic_devices_df['subpart'] = 'SubpartW'

df = pd.concat([df,pneumatic_devices_df], ignore_index=True)
df = df.drop(columns=['state'])

## Concat AssociatedGasVentingFlaring from detailed dataset

In [ ]:
len(df[df.reporting_category == 'AssociatedGasVentingFlaring'])

In [ ]:
# Drop AssociatedGasVentingFlaring from original dataset
df = df[df.reporting_category != 'AssociatedGasVentingFlaring']

In [ ]:
# add them from this csv file
venting_flaring_df = pd.read_csv('ef_w_associated_ng_units.csv')

In [ ]:
venting_flaring_df[['basin_ag_flaring_co2_emissions','basin_ag_venting_co2_emissions','venting_ch4_emissions','flaring_ch4_emissions']] = venting_flaring_df[['basin_ag_flaring_co2_emissions','basin_ag_venting_co2_emissions','venting_ch4_emissions','flaring_ch4_emissions']].fillna(0)
venting_flaring_df['total_co2_emissions'] =  venting_flaring_df['basin_ag_flaring_co2_emissions'] + venting_flaring_df['basin_ag_venting_co2_emissions']
venting_flaring_df['total_ch4_emissions'] =  venting_flaring_df['venting_ch4_emissions'] + venting_flaring_df['flaring_ch4_emissions']
venting_flaring_df = venting_flaring_df.drop(['basin_ag_flaring_co2_emissions','basin_ag_venting_co2_emissions','venting_ch4_emissions','flaring_ch4_emissions'],axis=1)
venting_flaring_df = venting_flaring_df.rename(columns={'flaring_n2o_emissions':'total_n2o_emissions'})
venting_flaring_df[['total_n2o_emissions']] = venting_flaring_df[['total_n2o_emissions']].fillna(0)
venting_flaring_df['total_ch4_emissions'] = venting_flaring_df['total_ch4_emissions'] * 25
venting_flaring_df['total_n2o_emissions'] = venting_flaring_df['total_n2o_emissions'] * 298
venting_flaring_df['reporting_category'] = 'AssociatedGasVentingFlaring'
venting_flaring_df['subpart'] = 'SubpartW'
venting_flaring_df = venting_flaring_df.drop(['facility_name','industry_segment','wells_two_year_delay'], axis=1)
df = pd.concat([df,venting_flaring_df], ignore_index=True)

## Concat AtmosphericTanks from detailed dataset

In [ ]:
len(df[df.reporting_category == 'AtmosphericTanks'])

In [ ]:
# Drop AssociatedGasVentingFlaring from original dataset
df = df[df.reporting_category != 'AtmosphericTanks']

In [ ]:
# add them from this csv file
athmospheric_tanks_df = pd.read_csv('ef_w_atm_stg_tanks_calc1or2.csv')

In [ ]:
athmospheric_tanks_df[['dump_valve_co2_emissions','flaring_co2_emissions','vapor_recovery_co2_emissions','venting_co2_emissions']] = athmospheric_tanks_df[['dump_valve_co2_emissions','flaring_co2_emissions','vapor_recovery_co2_emissions','venting_co2_emissions']].fillna(0)
athmospheric_tanks_df[['dump_valve_ch4_emissions','flaring_ch4_emissions','vapor_recovery_ch4_emissions','venting_ch4_emissions']] = athmospheric_tanks_df[['dump_valve_ch4_emissions','flaring_ch4_emissions','vapor_recovery_ch4_emissions','venting_ch4_emissions']].fillna(0)

athmospheric_tanks_df['total_co2_emissions'] =  athmospheric_tanks_df['dump_valve_co2_emissions'] + athmospheric_tanks_df['flaring_co2_emissions'] + athmospheric_tanks_df['vapor_recovery_co2_emissions'] + athmospheric_tanks_df['venting_co2_emissions']
athmospheric_tanks_df['total_ch4_emissions'] =  athmospheric_tanks_df['dump_valve_ch4_emissions'] + athmospheric_tanks_df['flaring_ch4_emissions'] + athmospheric_tanks_df['vapor_recovery_ch4_emissions']+athmospheric_tanks_df['venting_ch4_emissions'] 
athmospheric_tanks_df = athmospheric_tanks_df.rename(columns={'flaring_n2o_emissions':'total_n2o_emissions'})
athmospheric_tanks_df[['total_n2o_emissions']] = athmospheric_tanks_df[['total_n2o_emissions']].fillna(0)
athmospheric_tanks_df = athmospheric_tanks_df.drop(['dump_valve_co2_emissions','flaring_co2_emissions','vapor_recovery_co2_emissions','venting_co2_emissions'],axis=1)
athmospheric_tanks_df = athmospheric_tanks_df.drop(['dump_valve_ch4_emissions','flaring_ch4_emissions','vapor_recovery_ch4_emissions','venting_ch4_emissions'],axis=1)
athmospheric_tanks_df['total_ch4_emissions'] = athmospheric_tanks_df['total_ch4_emissions'] * 25
athmospheric_tanks_df['total_n2o_emissions'] = athmospheric_tanks_df['total_n2o_emissions'] * 298
athmospheric_tanks_df=athmospheric_tanks_df[['facility_id','reporting_year','total_co2_emissions','total_ch4_emissions','total_n2o_emissions','annual_ch4_recovered','annual_co2_recovered','atmospheric_tank_count',
                                             'average_a_p_i_gravity','average_separator_pressure','average_separator_temperature',
                                             'hrs_dump_valves_did_not_close','not_on_well_pad_tank_count','num_seps_with_stuck_dump_val',
                                             'number_of_wells_send_oil','tanks_venting_to_atm','tanks_with_flaring','tanks_with_vapor_recovery',
'total_volume_of_oil']]
athmospheric_tanks_df['reporting_category'] = 'AtmosphericTanks'
athmospheric_tanks_df['subpart'] = 'SubpartW'

In [ ]:
athmospheric_tanks_2_df = pd.read_csv('ef_w_atm_stg_tanks_calc3.csv')

athmospheric_tanks_2_df[['co2_emissions_from_flare','co2_emissions_not_flared','ch4_emissions_from_flare','ch4_emissions_not_flared']] = athmospheric_tanks_2_df[['co2_emissions_from_flare','co2_emissions_not_flared','ch4_emissions_from_flare','ch4_emissions_not_flared']].fillna(0)

athmospheric_tanks_2_df['total_co2_emissions'] =  athmospheric_tanks_2_df['co2_emissions_from_flare'] + athmospheric_tanks_2_df['co2_emissions_not_flared']
athmospheric_tanks_2_df['total_ch4_emissions'] =  athmospheric_tanks_2_df['ch4_emissions_from_flare'] + athmospheric_tanks_2_df['ch4_emissions_not_flared']
athmospheric_tanks_2_df = athmospheric_tanks_2_df.drop(['co2_emissions_from_flare','co2_emissions_not_flared','ch4_emissions_from_flare','ch4_emissions_not_flared'],axis=1)
athmospheric_tanks_2_df = athmospheric_tanks_2_df.rename(columns={'n2o_emissions_from_flare':'total_n2o_emissions'})
athmospheric_tanks_2_df[['total_n2o_emissions']] = athmospheric_tanks_2_df[['total_n2o_emissions']].fillna(0)
athmospheric_tanks_2_df['total_ch4_emissions'] = athmospheric_tanks_2_df['total_ch4_emissions'] * 25
athmospheric_tanks_2_df['total_n2o_emissions'] = athmospheric_tanks_2_df['total_n2o_emissions'] * 298
athmospheric_tanks_2_df=athmospheric_tanks_2_df[['facility_id','reporting_year','total_co2_emissions','total_ch4_emissions','total_n2o_emissions',
                                                 'annual_oil_throughput','atmospheric_tank_count','fract_oil_throughput_flaring',
                                                'fract_throughput_with_vapor','tanks_with_flares','tanks_without_flares',
                                                'wells_with_seps','wells_without_seps']]
athmospheric_tanks_2_df['reporting_category'] = 'AtmosphericTanks'
athmospheric_tanks_2_df['subpart'] = 'SubpartW'

In [ ]:
df = pd.concat([df,athmospheric_tanks_df,athmospheric_tanks_2_df], ignore_index=True)
len(df)

In [ ]:
generate_column_info(df)

In [ ]:
df[(df['facility_id']==1001727) & df['reporting_year']==2023]

## Create total_ghg_emissions column

In [ ]:
df['total_ghg_emissions'] = df['total_co2_emissions'] + df['total_ch4_emissions'] + df['total_n2o_emissions']
df = df[df['total_ghg_emissions']>0]

In [ ]:
len(df)

In [ ]:
generate_column_info(df)

In [ ]:
df[(df['reporting_year']==2022) & (df['facility_id']==1000435)][['reporting_category','total_co2_emissions','total_ch4_emissions','total_n2o_emissions','total_ghg_emissions']]

## Filter df to only required subsectors

In [ ]:
facilities_in_required_subsectors = facilities_master_df['facility_id'].unique()
print(len(df))
df = df[df.facility_id.isin(facilities_in_required_subsectors)]
print(len(df))

## Join Facilities Metadata

In [ ]:
df = pd.merge(df,facilities_master_df,how='left',on=['facility_id','reporting_year'])


In [ ]:
# remove rows for facility ids which do not have an exact match for the facilityid-reporting year combination in the facilities
#master
df = df[df['sub_sector'].notnull()]#.groupby(['facility_id','reporting_year']).count().reset_index()


In [ ]:
generate_column_info(df)

## Remove Rows with Null Reporting Category

In [ ]:
#df['facility_id'] = df['facility_id'].astype('int')
#df['reporting_year'] = df['reporting_year'].astype('int')
#df[(df['facility_id']==1002685) & (df['reporting_year']==2023)]
df[df['reporting_category'].isna()]

In [ ]:
df = df[df['reporting_category'].notnull()]

In [ ]:
#df[df['reporting_category'].isna()]

## Filter rows based on EPA verified facility_id reporting_year list

In [ ]:
#epa_verified_unique_facilities_df['EPA verified'] = 1
df = pd.merge(df,epa_verified_unique_facilities_df,how='left',on=['facility_id','reporting_year'])
df

In [ ]:
df[df['EPA_verified']==1]

In [ ]:
#df = df[df['GHG FACILITY_WIDE_GHG_QUANTITY']>0]

## Validate GHG Quantity for facility and add diff_percent_GHG to df

In [ ]:

validation_df = df
validation_df = validation_df[['facility_id','reporting_year','total_co2_emissions','total_ch4_emissions','total_n2o_emissions','total_ghg_emissions']]
validation_df=validation_df.groupby(['facility_id','reporting_year']).sum().reset_index()
validation_df = pd.merge(validation_df,facilities_master_df[['facility_id','reporting_year','FACILITY_WIDE_GHG_QUANTITY']],how='left',on=['facility_id','reporting_year'])
validation_df['diff'] = (validation_df['FACILITY_WIDE_GHG_QUANTITY']-validation_df['total_ghg_emissions']).abs()
validation_df['diff_percent_GHG']=100*validation_df['diff']/validation_df['FACILITY_WIDE_GHG_QUANTITY']
validation_df.to_csv('output_validation.csv', index=False)
validation_df = validation_df[['facility_id','reporting_year','diff_percent_GHG']]
#validation_df[validation_df.diff_percent_GHG > 10]

In [ ]:
df = pd.merge(df,validation_df,how='left',on=['facility_id','reporting_year'])

In [ ]:
generate_column_info(df)

In [ ]:
df.to_csv('output.csv', index=False)

In [ ]:
#AssociatedGasVentingFlaring

# AtmosphericTanks